In [2]:
import pandas as pd
import numpy as np

In [3]:
data_from_scrapping = pd.read_csv('stat_questions.csv')
data_from_scrapping.head()

,web-scraper-order,web-scraper-start-url,questions_and_answers
0,1675270905-1,https://intellipaat.com/blog/interview-questio...,1. How is the statistical significance of an i...
1,1675270905-2,https://intellipaat.com/blog/interview-questio...,Hypothesis testing is used to find out the sta...
2,1675270905-3,https://intellipaat.com/blog/interview-questio...,"After calculating the p-value, the null hypoth..."
3,1675270905-4,https://intellipaat.com/blog/interview-questio...,NaN
4,1675270905-5,https://intellipaat.com/blog/interview-questio...,2. Where are long-tailed distributions used?


In [4]:
data_droped = data_from_scrapping.drop(['web-scraper-order', 'web-scraper-start-url'], axis=1).copy()
data_droped.head()

,questions_and_answers
0,1. How is the statistical significance of an i...
1,Hypothesis testing is used to find out the sta...
2,"After calculating the p-value, the null hypoth..."
3,NaN
4,2. Where are long-tailed distributions used?


In [7]:
data_droped.dropna(how='all', inplace=True)
data_droped.reset_index(drop=True, inplace=True)
data_droped_string = data_droped.convert_dtypes(convert_string=True)
data_droped_string.info()

# converting the column to array:
qa_array = data_droped_string['questions_and_answers'].to_numpy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 1 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   questions_and_answers  272 non-null    string
dtypes: string(1)
memory usage: 2.2 KB


In [8]:
qa_array

array(['1. How is the statistical significance of an insight assessed?',
       'Hypothesis testing is used to find out the statistical significance of the insight. To elaborate, the null hypothesis and the alternate hypothesis are stated, and the p-value is calculated.',
       'After calculating the p-value, the null hypothesis is assumed true, and the values are determined. To fine-tune the result, the alpha value, which denotes the significance, is tweaked. If the p-value turns out to be less than the alpha, then the null hypothesis is rejected. This ensures that the result obtained is statistically significant.',
       '2. Where are long-tailed distributions used?',
       'A long-tailed distribution is a type of distribution where the tail drops off gradually toward the end of the curve.',
       'The Pareto principle and the product sales distribution are good examples to denote the use of long-tailed distributions. Also, it is widely used in classification and regression probl

In [32]:
rows_with_digits = []
rows_without_digits = []
answer_list = []
for i in qa_array:
    if not i[0].isdigit():
        rows_without_digits.append(i)
    else:
        # Joining the answers stored in list into single string and clearing the list, for next answers
        # matching the question
        if rows_without_digits:
            result_answer = ''.join(rows_without_digits)
            answer_list.append(result_answer)
            rows_without_digits = []
        rows_with_digits.append(i)
# Last if to make sure the last separated answer is also added to the list
if rows_without_digits:
    result_answer = ''.join(rows_without_digits)
    answer_list.append(result_answer)
        
#print(f'Rows with digits: {rows_with_digits}\n')
#print(f'List of answers: {answer_list}')

# Saving the results to the data frame with the questions in one column an the answers in another:
# Checking if the answers and question list are  matching in lenght:
try:
    if len(rows_with_digits) == len(answer_list):
        # creating the dict for pandas:
        qa_dict_to_df = {'questions': rows_with_digits, 'answers': answer_list}
except ValueError: 
    print("Lengths of provieded data frame differ in length, separation faild")

try:
    # creating df from dict
    df_separated_qa = pd.DataFrame(qa_dict_to_df)
except (TypeError, KeyError): 
    print("Creating of dataframe with answer, failed, check the dict used to creating the df, or the lists with answers")


In [34]:
df_separated_qa.tail()

,questions,answers
68,69. What is the impact of outliers in statistics?,Outliers in statistics have a very negative im...
69,"70. When creating a statistical model, how do ...",Overfitting can be detected by cross-validatio...
70,71. What is a survivorship bias?,The survivorship bias is the flaw of the sampl...
71,72. What is an undercoverage bias?,The undercoverage bias is a bias that occurs w...
72,74. What is the relationship between standard ...,Standard deviation is the square root of stand...
